In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


import sklearn

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC


import xgboost as xgb


from sklearn import tree
from tensorflow import keras

In [21]:
class Preprocessing():
    
    def get_unique(self,dataset,columns):
        for item in columns:
            print(dataset[item].unique())
        print(dataset.isnull().sum())
    
    def pre_processing(self,dataset):
        #print(dataset.isnull().sum())
        dataset['gender'] = dataset['gender'].map({'m':1,'f':0})
        dataset['education'] = dataset['education'].map({"Master's & above":0,"Bachelor's":1,
                                                        "Below Secondary":2})
        dataset['department'] = dataset['department'].map({'Sales & Marketing':0,'Operations':1,
                                                           'Technology':2,'Analytics':3,'R&D':4,
                                                           'Procurement':5,'Finance':6,'HR':7,'Legal':8})
        
        dataset['recruitment_channel'] = dataset['recruitment_channel'].map({'sourcing':0,'other':1,
                                                                             'referred':2})
        return dataset
    
    def adjust_null(self,dataset):
        
        dataset['education'].fillna(dataset['education'].mode()[0],inplace=True)
        dataset['previous_year_rating'].fillna(dataset['previous_year_rating'].mean(),inplace=True)
        
        #print(dataset.isnull().sum())
        return dataset
        
        
    def get_preprocess_data(self,data):
        dataset = self.pre_processing(data)
        dataset = self.adjust_null(dataset)
        #print(dataset.head())
        #print(len(dataset))
        dataset = dataset.dropna(axis=0)
        dataset = dataset.drop(['employee_id','region'],axis=1)

        #print(dataset.isnull().sum())
        #print(len(dataset))
        self.train,self.test = train_test_split(dataset,test_size = 0.1,random_state = 0,shuffle=False)
        
        self.y_train = self.train['is_promoted']
        self.x_train = self.train.drop('is_promoted',axis=1)
        
        
        self.y_test = self.test['is_promoted']
        self.x_test = self.test.drop('is_promoted',axis=1)
        return dataset
        #print(dataset.describe())

In [155]:
class GenModel():
    def __init__(self):
        self.model = None
        self.x_train = preprocess_obj.x_train
        self.y_train = preprocess_obj.y_train
        self.x_test = preprocess_obj.x_test
        self.y_test = preprocess_obj.y_test
        
    def train(self):
        self.model1 = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
        
        self.model2 = GradientBoostingClassifier(learning_rate=0.01,random_state=1)

        self.model3 = LogisticRegression(class_weight='balanced')
        
        #self.model4 = xgb.XGBClassifier(random_state=1,learning_rate=0.01)
        self.model4 = xgb.XGBClassifier(random_state=10, n_estimators=100, max_depth=1)

        self.model5 = AdaBoostClassifier(random_state=1,n_estimators=5)

        self.model6 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        
        self.model7 = MLPClassifier(hidden_layer_sizes=(3), max_iter=100)
        
        self.model8 = RandomForestClassifier(random_state=1, max_depth=10, n_estimators=10)

        self.model1.fit(self.x_train,self.y_train)
        print("Bagging: ")
        print(self.model1.score(self.x_test,self.y_test))
        print("\n")
        
        self.model2.fit(self.x_train,self.y_train)
        print("Gradient Boosting: ")
        print(self.model2.score(self.x_test,self.y_test))
        self.model2.fit(self.x_test,self.y_test)
        print(self.model2.score(self.x_test,self.y_test))
        print("\n")
        
        self.model3.fit(self.x_train,self.y_train)
        print("Logistic Regression: ")
        print(self.model3.score(self.x_test,self.y_test))
        print("\n")
        
        print("XGB: ")
        self.model4.fit(self.x_train,self.y_train)
        print(self.model4.score(self.x_test,self.y_test))
        self.model4.fit(self.x_test,self.y_test)
        print(self.model4.score(self.x_test,self.y_test))
        print("\n")
        
        print("AdaBoost: ")
        self.model5.fit(self.x_train,self.y_train)
        print(self.model5.score(self.x_test,self.y_test))
        self.model5.fit(self.x_test,self.y_test)
        print(self.model5.score(self.x_test,self.y_test))
        print("\n")
        
        print("KNN: ")
        self.model6.fit(self.x_train,self.y_train)
        print(self.model6.score(self.x_test,self.y_test))
        print("\n")
        
        print("MLP: ")
        self.model7.fit(self.x_train,self.y_train)
        print(self.model7.score(self.x_test,self.y_test))
        print("\n")
        
        print("Random Forest: ")
        self.model8.fit(self.x_train,self.y_train)
        print(self.model8.score(self.x_test,self.y_test))
        
    def nn_model1(self):
        n_cols = self.x_train.shape[1]
        self.model = keras.models.Sequential()
    
        self.model.add(keras.layers.Dense(256, kernel_initializer="truncated_normal", activation = "relu", input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
    
        self.model.add(keras.layers.Dense(128, kernel_initializer="truncated_normal", activation = "relu", input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
    
        self.model.add(keras.layers.Dense(64, kernel_initializer="truncated_normal", activation = "relu", input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
    
        self.model.add(keras.layers.Dense(1, kernel_initializer="truncated_normal", activation = "sigmoid" ))
    
        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ["acc"])
        self.model.fit(self.x_train,self.y_train, batch_size = 128, epochs=200, validation_data = (self.x_test,self.y_test))
    
    def nn_model2(self):
        n_cols = self.x_train.shape[1]
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.Dense(128, activation='relu', input_shape = (n_cols,)))
        self.model.add(keras.layers.Dropout(0.1))
        self.model.add(keras.layers.Dense(64, activation='relu'))
        self.model.add(keras.layers.Dropout(0.1))
        self.model.add(keras.layers.Dense(32, activation='relu'))
        self.model.add(keras.layers.Dense(1, activation='sigmoid'))
        self.model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
        self.model.fit(self.x_train,self.y_train, batch_size = 32,epochs=50, validation_data = (self.x_test,self.y_test))
        

In [156]:
train_data = pd.read_csv(os.path.join('Data','train.csv'))
test_data = pd.read_csv(os.path.join('Data','train.csv'))
preprocess_obj = Preprocessing()
#preprocess_obj.get_unique(train_data,['department','region','education','gender','recruitment_channel'])
train_data = preprocess_obj.get_preprocess_data(train_data)

In [157]:
model_obj = GenModel()

In [ ]:
model_obj.train()

In [158]:
model_obj.nn_model1()

Train on 49327 samples, validate on 5481 samples
Epoch 1/200
49327/49327 [==============================] - 1s 26us/sample - loss: 0.2871 - acc: 0.9145 - val_loss: 0.2670 - val_acc: 0.9111
Epoch 2/200
49327/49327 [==============================] - 1s 18us/sample - loss: 0.2563 - acc: 0.9154 - val_loss: 0.2588 - val_acc: 0.9126
Epoch 3/200
49327/49327 [==============================] - 1s 18us/sample - loss: 0.2480 - acc: 0.9173 - val_loss: 0.2485 - val_acc: 0.9164
Epoch 4/200
49327/49327 [==============================] - 1s 19us/sample - loss: 0.2400 - acc: 0.9197 - val_loss: 0.2431 - val_acc: 0.9159
Epoch 5/200
49327/49327 [==============================] - 1s 25us/sample - loss: 0.2307 - acc: 0.9222 - val_loss: 0.2321 - val_acc: 0.9201
Epoch 6/200
49327/49327 [==============================] - 1s 24us/sample - loss: 0.2250 - acc: 0.9230 - val_loss: 0.2358 - val_acc: 0.9179
Epoch 7/200
49327/49327 [==============================] - 1s 18us/sample - loss: 0.2200 - acc: 0.9253 - val_lo

Epoch 59/200
49327/49327 [==============================] - 1s 24us/sample - loss: 0.1816 - acc: 0.9370 - val_loss: 0.1994 - val_acc: 0.9298
Epoch 60/200
49327/49327 [==============================] - 1s 25us/sample - loss: 0.1799 - acc: 0.9378 - val_loss: 0.1997 - val_acc: 0.9305
Epoch 61/200
49327/49327 [==============================] - 1s 24us/sample - loss: 0.1825 - acc: 0.9361 - val_loss: 0.1872 - val_acc: 0.9347
Epoch 62/200
49327/49327 [==============================] - 2s 35us/sample - loss: 0.1787 - acc: 0.9384 - val_loss: 0.2213 - val_acc: 0.9254
Epoch 63/200
49327/49327 [==============================] - 2s 31us/sample - loss: 0.1791 - acc: 0.9379 - val_loss: 0.2294 - val_acc: 0.9241
Epoch 64/200
49327/49327 [==============================] - 1s 20us/sample - loss: 0.1793 - acc: 0.9376 - val_loss: 0.1976 - val_acc: 0.9316
Epoch 65/200
49327/49327 [==============================] - 1s 20us/sample - loss: 0.1777 - acc: 0.9389 - val_loss: 0.2021 - val_acc: 0.9319
Epoch 66/200


Epoch 117/200
49327/49327 [==============================] - 1s 28us/sample - loss: 0.1735 - acc: 0.9394 - val_loss: 0.2048 - val_acc: 0.9301
Epoch 118/200
49327/49327 [==============================] - 1s 26us/sample - loss: 0.1733 - acc: 0.9400 - val_loss: 0.1972 - val_acc: 0.9321
Epoch 119/200
49327/49327 [==============================] - 1s 23us/sample - loss: 0.1741 - acc: 0.9395 - val_loss: 0.1979 - val_acc: 0.9312
Epoch 120/200
49327/49327 [==============================] - 1s 24us/sample - loss: 0.1759 - acc: 0.9391 - val_loss: 0.2053 - val_acc: 0.9309
Epoch 121/200
49327/49327 [==============================] - 1s 23us/sample - loss: 0.1740 - acc: 0.9398 - val_loss: 0.2008 - val_acc: 0.9288
Epoch 122/200
49327/49327 [==============================] - 1s 29us/sample - loss: 0.1730 - acc: 0.9396 - val_loss: 0.2011 - val_acc: 0.9285
Epoch 123/200
49327/49327 [==============================] - 1s 25us/sample - loss: 0.1734 - acc: 0.9396 - val_loss: 0.1967 - val_acc: 0.9303
Epoch 

Epoch 175/200
49327/49327 [==============================] - 1s 20us/sample - loss: 0.1725 - acc: 0.9395 - val_loss: 0.2028 - val_acc: 0.9301
Epoch 176/200
49327/49327 [==============================] - 1s 21us/sample - loss: 0.1723 - acc: 0.9400 - val_loss: 0.1974 - val_acc: 0.9321
Epoch 177/200
49327/49327 [==============================] - 1s 25us/sample - loss: 0.1725 - acc: 0.9395 - val_loss: 0.2026 - val_acc: 0.9299
Epoch 178/200
49327/49327 [==============================] - 1s 23us/sample - loss: 0.1720 - acc: 0.9398 - val_loss: 0.2071 - val_acc: 0.9287
Epoch 179/200
49327/49327 [==============================] - 1s 21us/sample - loss: 0.1729 - acc: 0.9393 - val_loss: 0.2028 - val_acc: 0.9305
Epoch 180/200
49327/49327 [==============================] - 1s 22us/sample - loss: 0.1726 - acc: 0.9397 - val_loss: 0.2110 - val_acc: 0.9283
Epoch 181/200
49327/49327 [==============================] - 1s 22us/sample - loss: 0.1701 - acc: 0.9407 - val_loss: 0.2125 - val_acc: 0.9292
Epoch 

In [162]:
class Prediction():
    def __init__(self):
        self.model = model_obj.model
        self.use_nn = True
    def adjust_null(self,train,test):
        
        test['education'].fillna(train['education'].mode()[0],inplace=True)
        test['previous_year_rating'].fillna(train['previous_year_rating'].mean(),inplace=True)
        return test
   
    def prediction(self,test_data):
        emp_id = test_data['employee_id']

        test_data = test_data.drop(['employee_id','region'],axis=1)
        test_data = preprocess_obj.pre_processing(test_data)
        test_data = self.adjust_null(train_data,test_data)
        print(test_data.isnull().sum())
        print(test_data.head())
        
        if self.use_nn == True:
            y_predict=self.model.predict_classes(test_data)
            y_predict = y_predict.flatten()
            print(y_predict[1000:2000])
            #y_predict = np.argmax(y_predict,axis=1)
            #print(y_predict[1000:2000])
        else:
            y_predict=self.model.predict(test_data)
            
        print(emp_id.head())
        print(y_predict[:10])
        submission = pd.DataFrame({'employee_id': emp_id,'is_promoted': y_predict})

        submission.head()
        #y_predict=pd.DataFrame(y_predict,columns =['Loan_Status'],index=test_data.index)
        submission.to_csv("Submission.csv",index=False)
        print(submission.head())

In [163]:
test_data = pd.read_csv(os.path.join('Data','test.csv')) 
#preprocess_obj.get_unique(test_data,['department','region','education','gender','recruitment_channel'])
test_data.isnull().sum()
#test_data['Dependents'].unique() 
#pd.unique(test_data[['Dependents', 'Loan_Amount_Term']].values)

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [164]:
pred_obj = Prediction()
pred_obj.prediction(test_data)

department              0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64
   department  education  gender  recruitment_channel  no_of_trainings  age  \
0           2        1.0       1                    0                1   24   
1           7        1.0       0                    1                1   31   
2           0        1.0       1                    1                1   31   
3           5        1.0       0                    1                3   31   
4           6        1.0       1                    0                1   30   

   previous_year_rating  length_of_service  KPIs_met >80%  awards_won?  \
0              3.329256                  1              1            0   
1              3.000000                  5              0            0   
2   